# Results Analysis 

In [34]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import re
pd.options.mode.chained_assignment = None  # 'warn' or None for default behavior
sns.set(style="whitegrid")

In [35]:
results_file = 'results/models_comparison/models_comparison_full.csv'
# results_file = 'models_comparison_new.csv'
df = pd.read_csv(results_file)
df = df[~df['Architecture'].str.contains('inat') & ~df['Architecture'].str.contains('CLIP_finetuned')]
df

,Architecture,Accuracy,Accuracy_TS,AUROC,AUROC_TS,ECE_15,ECE_15_TS,MCE_15,MCE_15_TS,Gamma Correlation,...,hAURC_selective_hier_TS,hAURC_climbing_hier_TS,hAURC_jumping_hier_TS,hAURC_maxcoverage_hier_TS,Gain_climbing_01_TS,Gain_jumping_01_TS,Gain_maxcoverage_01_TS,Gain_climbing_hier_TS,Gain_jumping_hier_TS,Gain_maxcoverage_hier_TS
0,alexnet_torchvision,56.554,56.50222,0.848762,0.847575,0.020001,0.020148,0.045109,0.045058,0.697524,...,0.085709,0.090885,0.094387,0.094281,-5.727459,-11.179369,-10.581677,-6.038769,-10.124355,-10.000549
1,bat_resnext26ts.ch_in1k,78.260,78.30667,0.852646,0.866543,0.152277,0.025664,0.207609,0.075454,0.705293,...,0.028076,0.025448,0.025605,0.025598,11.508905,10.902103,10.953860,9.360756,8.803483,8.827900
2,beit_base_patch16_224.in22k_ft_in22k_in1k,85.220,85.21778,0.865260,0.873466,0.073966,0.024624,0.131687,0.129022,0.730520,...,0.015281,0.012819,0.012854,0.012853,18.816811,18.615511,18.629862,16.111869,15.884137,15.890001
3,beit_base_patch16_384.in22k_ft_in22k_in1k,86.820,86.75111,0.863012,0.872034,0.074450,0.029266,0.248498,0.209795,0.726025,...,0.013346,0.011005,0.011035,0.011033,20.210467,20.030504,20.042936,17.538852,17.320547,17.330080
4,beit_large_patch16_224.in22k_ft_in22k_in1k,87.478,87.39778,0.870189,0.878184,0.035049,0.031331,0.308009,0.122774,0.740379,...,0.011929,0.009849,0.009857,0.009857,20.083620,20.025284,20.029237,17.431484,17.368419,17.371066
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1111,xcit_tiny_24_p16_224.fb_in1k,79.452,79.33778,0.861033,0.865585,0.092239,0.036270,0.119930,0.195569,0.722066,...,0.026006,0.023484,0.023572,0.023574,12.447076,12.059244,12.078235,9.699151,9.361898,9.352548
1112,xcit_tiny_24_p16_384.fb_dist_in1k,82.564,82.55556,0.860989,0.864563,0.030187,0.036511,0.060142,0.118815,0.721977,...,0.020049,0.017860,0.017894,0.017894,14.082824,13.894001,13.909741,10.918618,10.749358,10.751415
1113,xcit_tiny_24_p8_224.fb_dist_in1k,82.590,82.53778,0.860752,0.864788,0.034570,0.037626,0.436421,0.131205,0.721505,...,0.020108,0.017912,0.017963,0.017963,13.754816,13.512859,13.523234,10.922758,10.668865,10.670624
1114,xcit_tiny_24_p8_224.fb_in1k,81.898,81.81778,0.861597,0.867529,0.082945,0.034863,0.187275,0.124365,0.723193,...,0.021494,0.018840,0.018889,0.018889,15.057658,14.832755,14.846491,12.346108,12.121312,12.121122


Calculate Hierarchical Gain

In [36]:
# No TS <-> No TS, TS <-> TS
for inf_rule in ['selective', 'maxcoverage', 'climbing']:
    for suffix in ['hier', 'hier_TS', '01', '01_TS']:
        col_name = f'hAURC_{inf_rule}_{suffix}'
        df[col_name] = 1000*df[col_name]

for inf_rule in ['maxcoverage', 'climbing']:
    # for suffix in ['01', '01_TS']:
    for suffix in ['01', '01_TS', 'hier', 'hier_TS']:
        col_name = f'hAURC_{inf_rule}_{suffix}'
        df[f'Gain_{col_name}'] = 100*(df[f'hAURC_selective_{suffix}']-df[col_name])/df[f'hAURC_selective_{suffix}']

# Our algorithms <-> Baselines
for inf_rule in ['climbing']:
    col_name = f'hAURC_{inf_rule}_hier_TS'
    df[f'Gain_{inf_rule}_hier_TS_F'] = 100*(df[f'hAURC_selective_hier']-df[col_name])/df[f'hAURC_selective_hier']
    col_name = f'hAURC_{inf_rule}_01_TS'
    df[f'Gain_{inf_rule}_01_TS_F'] = 100*(df[f'hAURC_selective_01']-df[col_name])/df[f'hAURC_selective_01']

In [37]:
df

,Architecture,Accuracy,Accuracy_TS,AUROC,AUROC_TS,ECE_15,ECE_15_TS,MCE_15,MCE_15_TS,Gamma Correlation,...,Gain_hAURC_maxcoverage_01,Gain_hAURC_maxcoverage_01_TS,Gain_hAURC_maxcoverage_hier,Gain_hAURC_maxcoverage_hier_TS,Gain_hAURC_climbing_01,Gain_hAURC_climbing_01_TS,Gain_hAURC_climbing_hier,Gain_hAURC_climbing_hier_TS,Gain_climbing_hier_TS_F,Gain_climbing_01_TS_F
0,alexnet_torchvision,56.554,56.50222,0.848762,0.847575,0.020001,0.020148,0.045109,0.045058,0.697524,...,-7.530761,-10.581677,-6.875490,-10.000549,-3.741580,-5.727459,-3.822449,-6.038769,-6.091782,-6.151149
1,bat_resnext26ts.ch_in1k,78.260,78.30667,0.852646,0.866543,0.152277,0.025664,0.207609,0.075454,0.705293,...,-1.269206,10.953860,-5.632583,8.827900,1.061489,11.508905,-3.272132,9.360756,12.333511,16.052890
2,beit_base_patch16_224.in22k_ft_in22k_in1k,85.220,85.21778,0.865260,0.873466,0.073966,0.024624,0.131687,0.129022,0.730520,...,12.273574,18.629862,7.711865,15.890001,12.917202,18.816811,8.465580,16.111869,18.312468,21.808498
3,beit_base_patch16_384.in22k_ft_in22k_in1k,86.820,86.75111,0.863012,0.872034,0.074450,0.029266,0.248498,0.209795,0.726025,...,13.546625,20.042936,8.967889,17.330080,14.090126,20.210467,9.618976,17.538852,19.179159,23.082232
4,beit_large_patch16_224.in22k_ft_in22k_in1k,87.478,87.39778,0.870189,0.878184,0.035049,0.031331,0.308009,0.122774,0.740379,...,15.817651,20.029237,11.968331,17.371066,16.022719,20.083620,12.215884,17.431484,19.051702,22.627545
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1111,xcit_tiny_24_p16_224.fb_in1k,79.452,79.33778,0.861033,0.865585,0.092239,0.036270,0.119930,0.195569,0.722066,...,3.683384,12.078235,-1.034392,9.352548,4.658987,12.447076,-0.054599,9.699151,9.505887,13.281826
1112,xcit_tiny_24_p16_384.fb_dist_in1k,82.564,82.55556,0.860989,0.864563,0.030187,0.036511,0.060142,0.118815,0.721977,...,10.389239,13.909741,6.356125,10.751415,10.749423,14.082824,6.720738,10.918618,11.841582,15.282733
1113,xcit_tiny_24_p8_224.fb_dist_in1k,82.590,82.53778,0.860752,0.864788,0.034570,0.037626,0.436421,0.131205,0.721505,...,9.885755,13.523234,6.135098,10.670624,10.266112,13.754816,6.544722,10.922758,11.923287,14.841366
1114,xcit_tiny_24_p8_224.fb_in1k,81.898,81.81778,0.861597,0.867529,0.082945,0.034863,0.187275,0.124365,0.723193,...,7.078883,14.846491,2.283281,12.121122,7.733799,15.057658,3.025701,12.346108,12.712313,16.589648


hAURC and Gain results for inference rules

In [38]:
n = np.sqrt(len(df))
Valmadre = '{Valmadre}'
print(f'Selective & {df["hAURC_selective_01"].mean():.2f} $\pm$ {df["hAURC_selective_01"].std()/n:.2f} & - \\\\')
print(f"Max-Coverage \cite{Valmadre} & {df['hAURC_maxcoverage_01'].mean():.2f} $\pm$ {df['hAURC_maxcoverage_01'].std()/n:.2f} & {df['Gain_maxcoverage_01'].mean():.2f} $\pm$ {df['Gain_maxcoverage_01'].std()/n:.2f} \\\\")
print(f"Climbing (Ours) & {df['hAURC_climbing_01_TS'].mean():.2f} $\pm$ {df['hAURC_climbing_01_TS'].std()/n:.2f} & {df['Gain_climbing_01_TS_F'].mean():.2f} $\pm$ {df['Gain_climbing_01_TS_F'].std()/n:.2f} \\\\")

Selective & 42.25 $\pm$ 0.46 & - \\
Max-Coverage \cite{Valmadre} & 39.96 $\pm$ 0.53 & 6.88 $\pm$ 0.26 \\
Climbing (Ours) & 36.49 $\pm$ 0.47 & 14.95 $\pm$ 0.22 \\


hAURC results for inference rules - appendix (01)

In [39]:
n = np.sqrt(len(df))
Valmadre = '{Valmadre}'
# No temp scaling
print(f'& Selective & {df["hAURC_selective_hier"].mean():.2f} $\pm$ {df["hAURC_selective_01"].std()/n:.2f} & - \\\\')
print(f"& Max-Coverage \cite{Valmadre} & {df['hAURC_maxcoverage_01'].mean():.2f} $\pm$ {df['hAURC_maxcoverage_01'].std()/n:.2f} & {df['Gain_maxcoverage_01'].mean():.2f} $\pm$ {df['Gain_maxcoverage_01'].std()/n:.2f} & {df['hAURC_maxcoverage_01'].mean():.2f} $\pm$ {df['hAURC_maxcoverage_01'].std()/n:.2f} \\\\")
print(f"& Climbing (Ours) & {df['hAURC_climbing_01'].mean():.2f} $\pm$ {df['hAURC_climbing_01'].std()/n:.2f} & {df['Gain_climbing_01'].mean():.2f} $\pm$ {df['Gain_climbing_01'].std()/n:.2f} \\\\")
# table formatting
print('\midrule\multirow{4}{4em}{\\textbf{Temp. Scaling}}')
# Temp Scaling
print(f'& Selective & {df["hAURC_selective_01_TS"].mean():.2f} $\pm$ {df["hAURC_selective_01_TS"].std()/n:.2f} & - \\\\')
print(f"& Max-Coverage \cite{Valmadre} & {df['hAURC_maxcoverage_01_TS'].mean():.2f} $\pm$ {df['hAURC_maxcoverage_01_TS'].std()/n:.2f} & {df['Gain_maxcoverage_01_TS'].mean():.2f} $\pm$ {df['Gain_maxcoverage_01_TS'].std()/n:.2f} \\\\")
print(f"& Climbing (Ours) & {df['hAURC_climbing_01_TS'].mean():.2f} $\pm$ {df['hAURC_climbing_01_TS'].std()/n:.2f} & {df['Gain_climbing_01_TS'].mean():.2f} $\pm$ {df['Gain_climbing_01_TS'].std()/n:.2f} \\\\")

& Selective & 24.98 $\pm$ 0.46 & - \\
& Max-Coverage \cite{Valmadre} & 39.96 $\pm$ 0.53 & 6.88 $\pm$ 0.26 & 39.96 $\pm$ 0.53 \\
& Climbing (Ours) & 39.21 $\pm$ 0.49 & 8.22 $\pm$ 0.19 \\
\midrule\multirow{4}{4em}{\textbf{Temp. Scaling}}
& Selective & 41.14 $\pm$ 0.46 & - \\
& Max-Coverage \cite{Valmadre} & 37.19 $\pm$ 0.52 & 11.05 $\pm$ 0.48 \\
& Climbing (Ours) & 36.49 $\pm$ 0.47 & 12.49 $\pm$ 0.16 \\


## Utils 

In [40]:
def combine_dfs(df, bs_df, repeat_bs=False):
    cols = ['Architecture', 'hAURC_climbing_01_TS', 'hAURC_climbing_01', 'hAURC_selective_01', 'Accuracy', 'Accuracy_TS', 'Gain_climbing_01_TS_F']
    df_combined = df[cols]
    if not repeat_bs:
        df_combined = df_combined.join(bs_df[cols].add_prefix('Baseline_'))
    else:
        df_combined['Baseline_Architecture'] = df['Baseline_Architecture']
        for i,row in df_combined.iterrows():
            bs_row = bs_df[bs_df['Architecture'] == row['Baseline_Architecture']]
            df_combined.loc[i, 'Baseline_Accuracy'] = bs_row['Accuracy'].values[0]
            df_combined.loc[i, 'Baseline_hAURC_selective_01'] = bs_row['hAURC_selective_01'].values[0]
            df_combined.loc[i, 'Baseline_hAURC_climbing_01_TS'] = bs_row['hAURC_climbing_01_TS'].values[0]
            df_combined.loc[i, 'Baseline_hAURC_climbing_01'] = bs_row['hAURC_climbing_01'].values[0]
            df_combined.loc[i, 'Baseline_Gain_climbing_01_TS_F'] = bs_row['Gain_climbing_01_TS_F'].values[0]
            
    df_combined['hAURC_improve_selective'] = 100*(df_combined['Baseline_hAURC_selective_01'] - df_combined['hAURC_selective_01'])/df_combined['Baseline_hAURC_selective_01']
    df_combined['hAURC_improve_climb_TS'] = 100*(df_combined['Baseline_hAURC_climbing_01_TS'] - df_combined['hAURC_climbing_01_TS'])/df_combined['Baseline_hAURC_climbing_01_TS']
    df_combined['hAURC_improve_climb'] = 100*(df_combined['Baseline_hAURC_climbing_01'] - df_combined['hAURC_climbing_01'])/df_combined['Baseline_hAURC_climbing_01']
    df_combined['Gain_climb_improve'] = 100*(df_combined['Gain_climbing_01_TS_F'] - df_combined['Baseline_Gain_climbing_01_TS_F'])/df_combined['Baseline_Gain_climbing_01_TS_F']
    df_combined['Accuracy_improve'] = 100*(df_combined['Accuracy_TS'] - df_combined['Baseline_Accuracy'])/df_combined['Baseline_Accuracy']
    df_combined['Accuracy_delta'] = df_combined['Accuracy_TS'] - df_combined['Baseline_Accuracy']
    return df_combined

## Train Methods Comparison

### Distillation

In [41]:
distilled = ['hrnet_w18_ssld.paddle_in1k', 'hrnet_w48_ssld.paddle_in1k', 'pit_ti_distilled_224.in1k', 'ecaresnet101d.miil_in1k','ecaresnet50d.miil_in1k', 'efficientnet_b1_pruned.in1k','efficientnet_b3_pruned.in1k','deit_base_distilled_patch16_224.fb_in1k', 'deit_base_distilled_patch16_384.fb_in1k', 'deit_small_distilled_patch16_224.fb_in1k', 'deit_tiny_distilled_patch16_224.fb_in1k', 'fastvit_ma36.apple_dist_in1k', 'fastvit_s12.apple_dist_in1k', 'fastvit_sa12.apple_dist_in1k', 'fastvit_sa24.apple_dist_in1k', 'fastvit_t12.apple_dist_in1k', 'fastvit_t8.apple_dist_in1k', 'pit_b_distilled_224.in1k', 'pit_s_distilled_224.in1k', 'pit_xs_distilled_224.in1k', 'resmlp_12_224.fb_distilled_in1k', 'resmlp_24_224.fb_distilled_in1k', 'resmlp_36_224.fb_distilled_in1k', 'resmlp_big_24_224.fb_distilled_in1k', 'resnetv2_50x1_bit.goog_distilled_in1k', 'tiny_vit_11m_224.dist_in22k_ft_in1k', 'tiny_vit_21m_224.dist_in22k_ft_in1k', 'tiny_vit_5m_224.dist_in22k_ft_in1k', 'xcit_large_24_p16_224.fb_dist_in1k', 'xcit_large_24_p8_224.fb_dist_in1k', 'xcit_medium_24_p16_224.fb_dist_in1k', 'xcit_medium_24_p8_224.fb_dist_in1k', 'xcit_nano_12_p16_224.fb_dist_in1k', 'xcit_nano_12_p8_224.fb_dist_in1k', 'xcit_small_12_p16_224.fb_dist_in1k', 'xcit_small_12_p8_224.fb_dist_in1k', 'xcit_small_24_p16_224.fb_dist_in1k', 'xcit_small_24_p8_224.fb_dist_in1k', 'xcit_tiny_12_p16_224.fb_dist_in1k', 'xcit_tiny_12_p8_224.fb_dist_in1k', 'xcit_tiny_24_p16_224.fb_dist_in1k', 'xcit_tiny_24_p8_224.fb_dist_in1k', ]
baselines = ['hrnet_w18.ms_in1k', 'hrnet_w48.ms_in1k', 'pit_ti_224', 'ecaresnet101d_pruned.miil_in1k','ecaresnet50d_pruned.miil_in1k','efficientnet_b1_torchvision','efficientnet_b3_torchvision','deit_base_patch16_224.fb_in1k', 'deit_base_patch16_384.fb_in1k', 'deit_small_patch16_224.fb_in1k', 'deit_tiny_patch16_224.fb_in1k', 'fastvit_ma36.apple_in1k', 'fastvit_s12.apple_in1k', 'fastvit_sa12.apple_in1k', 'fastvit_sa24.apple_in1k', 'fastvit_t12.apple_in1k', 'fastvit_t8.apple_in1k', 'pit_b_224.in1k', 'pit_s_224.in1k', 'pit_xs_224.in1k', 'resmlp_12_224.fb_in1k', 'resmlp_24_224.fb_in1k', 'resmlp_36_224.fb_in1k', 'resmlp_big_24_224.fb_in1k', 'resnetv2_50x1_bit.goog_in21k_ft_in1k', 'tiny_vit_11m_224.in1k', 'tiny_vit_21m_224.in1k', 'tiny_vit_5m_224.in1k', 'xcit_large_24_p16_224.fb_in1k', 'xcit_large_24_p8_224.fb_in1k', 'xcit_medium_24_p16_224.fb_in1k', 'xcit_medium_24_p8_224.fb_in1k', 'xcit_nano_12_p16_224.fb_in1k', 'xcit_nano_12_p8_224.fb_in1k', 'xcit_small_12_p16_224.fb_in1k', 'xcit_small_12_p8_224.fb_in1k', 'xcit_small_24_p16_224.fb_in1k', 'xcit_small_24_p8_224.fb_in1k', 'xcit_tiny_12_p16_224.fb_in1k', 'xcit_tiny_12_p8_224.fb_in1k', 'xcit_tiny_24_p16_224.fb_in1k', 'xcit_tiny_24_p8_224.fb_in1k', ]

dist_df = df[df['Architecture'].isin(distilled)].sort_values(by=['Architecture']).reset_index(drop=True)
dist_bs_df = df[df['Architecture'].isin(baselines)].sort_values(by=['Architecture']).reset_index(drop=True)
df_dist_combined = combine_dfs(dist_df, dist_bs_df)

In [42]:
df_dist_combined.sort_values(by='hAURC_improve_climb_TS', ascending=False)

,Architecture,hAURC_climbing_01_TS,hAURC_climbing_01,hAURC_selective_01,Accuracy,Accuracy_TS,Gain_climbing_01_TS_F,Baseline_Architecture,Baseline_hAURC_climbing_01_TS,Baseline_hAURC_climbing_01,Baseline_hAURC_selective_01,Baseline_Accuracy,Baseline_Accuracy_TS,Baseline_Gain_climbing_01_TS_F,hAURC_improve_selective,hAURC_improve_climb_TS,hAURC_improve_climb,Gain_climb_improve,Accuracy_improve,Accuracy_delta
31,xcit_medium_24_p8_224.fb_dist_in1k,24.358514,25.473224,28.691264,85.076,85.10000,15.101287,xcit_nano_12_p16_224.fb_in1k,68.427731,75.536756,75.662829,69.964,70.02222,9.562289,62.080106,64.402569,66.277048,57.925450,21.633983,15.13600
37,xcit_small_24_p8_224.fb_dist_in1k,24.818531,26.099464,29.461270,84.864,84.89111,15.758789,xcit_tiny_12_p16_224.fb_in1k,43.877580,48.810343,50.897770,77.132,77.17333,13.792726,42.116776,43.436874,46.528825,14.254344,10.059521,7.75911
26,tiny_vit_21m_224.dist_in22k_ft_in1k,22.599901,25.250940,28.644944,85.088,85.09333,21.103351,tiny_vit_5m_224.in1k,38.741833,49.545507,48.241892,79.168,79.17778,19.692549,40.622262,41.665381,49.034853,7.164140,7.484501,5.92533
15,hrnet_w48_ssld.paddle_in1k,27.504938,25.747985,32.359611,83.634,83.58667,15.002260,hrnet_w48.ms_in1k,37.926222,36.642363,43.762285,79.326,79.36444,13.335827,26.055939,27.477781,29.731648,12.495904,5.371089,4.26067
29,xcit_large_24_p8_224.fb_dist_in1k,24.025491,25.188992,28.505322,85.406,85.41778,15.715771,xcit_medium_24_p16_224.fb_in1k,32.728176,35.474514,37.972567,82.628,82.60444,13.810999,24.931803,26.590805,28.994115,13.791699,3.376313,2.78978
14,hrnet_w18_ssld.paddle_in1k,34.349445,32.419551,39.704670,81.088,81.02444,13.487644,hrnet_w18.ms_in1k,45.576768,44.306010,50.784164,76.756,76.57555,10.253976,21.816828,24.633872,26.828097,31.535756,5.561051,4.26844
23,resmlp_big_24_224.fb_distilled_in1k,26.800655,28.001992,31.863845,83.592,83.62667,15.890077,resnetv2_50x1_bit.goog_in21k_ft_in1k,34.124503,34.818187,41.124040,80.340,80.32222,17.020550,22.517718,21.462137,19.576536,-6.641810,4.090951,3.28667
7,efficientnet_b3_pruned.in1k,34.044798,38.656931,41.001619,80.852,80.83111,16.967187,efficientnet_b3_torchvision,41.846000,45.043094,48.560022,78.540,78.64000,13.826233,15.565073,18.642647,14.177896,22.717348,2.917125,2.29111
4,ecaresnet101d.miil_in1k,32.599454,40.280202,41.116189,82.160,82.16889,20.713824,ecaresnet101d_pruned.miil_in1k,39.563883,40.793019,45.176856,80.816,80.91111,12.424445,8.988379,17.602996,1.257121,66.718301,1.674037,1.35289
19,pit_xs_distilled_224.in1k,38.373537,39.149645,43.663315,79.166,79.18667,12.114926,resmlp_12_224.fb_in1k,46.231752,52.016306,53.355965,76.646,76.62666,13.352233,18.166011,16.997443,24.735824,-9.266672,3.314811,2.54067


### Imagenet21K/22K

In [43]:
pretrained_21_models = ['tresnet_m.miil_in21k_ft_in1k', 'tresnet_v2_l.miil_in21k_ft_in1k', 'maxvit_base_tf_384.in21k_ft_in1k', 'tf_efficientnetv2_b3.in21k_ft_in1k', 'tf_efficientnetv2_l.in21k_ft_in1k', 'tf_efficientnetv2_m.in21k_ft_in1k', 'tf_efficientnetv2_s.in21k_ft_in1k', 'tresnet_m.miil_in21k_ft_in1k', 'vit_base_patch16_224.augreg_in21k_ft_in1k', 'vit_base_patch16_384.augreg_in21k_ft_in1k', 'vit_base_patch32_224.augreg_in21k_ft_in1k', 'vit_base_patch32_384.augreg_in21k_ft_in1k', 'vit_small_patch16_224.augreg_in21k_ft_in1k', 'vit_small_patch16_384.augreg_in21k_ft_in1k', ]
pretrained_22_models = ['caformer_b36.sail_in22k_ft_in1k', 'caformer_b36.sail_in22k_ft_in1k_384', 'caformer_m36.sail_in22k_ft_in1k', 'caformer_m36.sail_in22k_ft_in1k_384', 'caformer_s18.sail_in22k_ft_in1k', 'caformer_s18.sail_in22k_ft_in1k_384', 'caformer_s36.sail_in22k_ft_in1k', 'caformer_s36.sail_in22k_ft_in1k_384', 'convformer_b36.sail_in22k_ft_in1k', 'convformer_b36.sail_in22k_ft_in1k_384', 'convformer_m36.sail_in22k_ft_in1k', 'convformer_m36.sail_in22k_ft_in1k_384', 'convformer_s18.sail_in22k_ft_in1k', 'convformer_s18.sail_in22k_ft_in1k_384', 'convformer_s36.sail_in22k_ft_in1k', 'convformer_s36.sail_in22k_ft_in1k_384', 'convnext_base.fb_in22k_ft_in1k', 'convnext_large.fb_in22k_ft_in1k', 'convnext_small.fb_in22k_ft_in1k', 'convnext_tiny.fb_in22k_ft_in1k', 'deit3_base_patch16_224.fb_in22k_ft_in1k', 'deit3_base_patch16_384.fb_in22k_ft_in1k', 'deit3_huge_patch14_224.fb_in22k_ft_in1k', 'deit3_large_patch16_224.fb_in22k_ft_in1k', 'deit3_large_patch16_384.fb_in22k_ft_in1k', 'deit3_medium_patch16_224.fb_in22k_ft_in1k', 'deit3_small_patch16_224.fb_in22k_ft_in1k', 'deit3_small_patch16_384.fb_in22k_ft_in1k', 'mobilevitv2_150.cvnets_in22k_ft_in1k', 'mobilevitv2_175.cvnets_in22k_ft_in1k', 'mobilevitv2_200.cvnets_in22k_ft_in1k', 'resmlp_big_24_224.fb_in22k_ft_in1k', 'swin_base_patch4_window12_384.ms_in22k_ft_in1k', 'swin_base_patch4_window7_224.ms_in22k_ft_in1k', 'swin_small_patch4_window7_224.ms_in22k_ft_in1k', 'swin_tiny_patch4_window7_224.ms_in22k_ft_in1k', 'tiny_vit_11m_224.dist_in22k_ft_in1k', 'tiny_vit_21m_224.dist_in22k_ft_in1k', 'tiny_vit_5m_224.dist_in22k_ft_in1k']
pretrained_21_models += pretrained_22_models
pretrained_21_baselines = ['tresnet_m.miil_in1k', 'tresnet_l.miil_in1k', 'maxvit_base_tf_384.in1k', 'tf_efficientnetv2_b3.in1k', 'tf_efficientnetv2_l.in1k', 'tf_efficientnetv2_m.in1k', 'tf_efficientnetv2_s.in1k', 'tresnet_m.miil_in1k', 'vit_base_patch16_224.augreg_in1k', 'vit_base_patch16_384.augreg_in1k', 'vit_base_patch32_224.augreg_in1k', 'vit_base_patch32_384.augreg_in1k', 'vit_small_patch16_224.augreg_in1k', 'vit_small_patch16_384.augreg_in1k' ]
pretrained_22_baselines = ['caformer_b36.sail_in1k', 'caformer_b36.sail_in1k_384', 'caformer_m36.sail_in1k', 'caformer_m36.sail_in1k_384', 'caformer_s18.sail_in1k', 'caformer_s18.sail_in1k_384', 'caformer_s36.sail_in1k', 'caformer_s36.sail_in1k_384', 'convformer_b36.sail_in1k', 'convformer_b36.sail_in1k_384', 'convformer_m36.sail_in1k', 'convformer_m36.sail_in1k_384', 'convformer_s18.sail_in1k', 'convformer_s18.sail_in1k_384', 'convformer_s36.sail_in1k', 'convformer_s36.sail_in1k_384', 'convnext_base.fb_in1k', 'convnext_large.fb_in1k', 'convnext_small.fb_in1k', 'convnext_tiny.fb_in1k', 'deit3_base_patch16_224.fb_in1k', 'deit3_base_patch16_384.fb_in1k', 'deit3_huge_patch14_224.fb_in1k', 'deit3_large_patch16_224.fb_in1k', 'deit3_large_patch16_384.fb_in1k', 'deit3_medium_patch16_224.fb_in1k', 'deit3_small_patch16_224.fb_in1k', 'deit3_small_patch16_384.fb_in1k', 'mobilevitv2_150.cvnets_in1k', 'mobilevitv2_175.cvnets_in1k', 'mobilevitv2_200.cvnets_in1k', 'resmlp_big_24_224.fb_in1k', 'swin_base_patch4_window12_384.ms_in1k', 'swin_base_patch4_window7_224.ms_in1k', 'swin_small_patch4_window7_224.ms_in1k', 'swin_tiny_patch4_window7_224.ms_in1k', 'tiny_vit_11m_224.in1k', 'tiny_vit_21m_224.in1k', 'tiny_vit_5m_224.in1k']
pretrained_21_baselines += pretrained_22_baselines

pre_21_df = df[df['Architecture'].isin(pretrained_21_models)].sort_values(by=['Architecture']).reset_index(drop=True)
pre_21_bs_df = df[df['Architecture'].isin(pretrained_21_baselines)].sort_values(by=['Architecture']).reset_index(drop=True)
pre_21_df_combined = combine_dfs(pre_21_df, pre_21_bs_df)

In [44]:
clip_ft_1k = {'convnext_base.clip_laion2b_augreg_ft_in1k': 'convnext_base.fb_in1k',
'eva_giant_patch14_336.clip_ft_in1k': 'eva_giant_patch14_336.m30m_ft_in22k_in1k',
'vit_base_patch16_clip_224.laion2b_ft_in1k': 'vit_base_patch16_224.augreg_in1k',
'vit_base_patch16_clip_224.openai_ft_in1k': 'vit_base_patch16_224.augreg_in1k',
'vit_base_patch16_clip_384.laion2b_ft_in1k': 'vit_base_patch16_384.augreg_in1k',
'vit_base_patch16_clip_384.openai_ft_in1k': 'vit_base_patch16_384.augreg_in1k',
'vit_base_patch32_clip_224.laion2b_ft_in1k': 'vit_base_patch32_224.augreg_in1k',
'vit_base_patch32_clip_224.openai_ft_in1k': 'vit_base_patch32_224.augreg_in1k'}

clip_ft_1k_df = df[df['Architecture'].isin(clip_ft_1k.keys())].sort_values(by=['Architecture']).reset_index(drop=True)
clip_ft_1k_bs_df = df[df['Architecture'].isin(clip_ft_1k.values())].sort_values(by=['Architecture']).reset_index(drop=True)
clip_ft_1k_df['Baseline_Architecture'] = clip_ft_1k_df.apply(lambda x: clip_ft_1k[x['Architecture']], axis=1)
clip_ft_1k_combined = combine_dfs(clip_ft_1k_df, clip_ft_1k_bs_df, repeat_bs=True)

### Finetuned on 12K 

In [45]:
clip_ft_12k = ['convnext_base.clip_laion2b_augreg_ft_in12k_in1k',  'vit_base_patch16_clip_224.laion2b_ft_in12k_in1k',  'vit_base_patch16_clip_224.openai_ft_in12k_in1k',  'vit_base_patch16_clip_384.laion2b_ft_in12k_in1k',  'vit_base_patch16_clip_384.openai_ft_in12k_in1k',  'vit_base_patch32_clip_224.laion2b_ft_in12k_in1k',  'vit_base_patch32_clip_384.laion2b_ft_in12k_in1k',  'vit_base_patch32_clip_384.openai_ft_in12k_in1k']
clip_ft_12k_baselines = ['convnext_base.fb_in1k', 'vit_base_patch16_224.augreg_in1k', 'vit_base_patch16_224.augreg_in1k', 'vit_base_patch16_384.augreg_in1k', 'vit_base_patch16_384.augreg_in1k', 'vit_base_patch32_224.augreg_in1k', 'vit_base_patch32_384.augreg_in1k', 'vit_base_patch32_384.augreg_in1k' ]
clip_ft_12k = {'convnext_base.clip_laion2b_augreg_ft_in12k_in1k':'convnext_base.fb_in1k',
'vit_base_patch16_clip_224.laion2b_ft_in12k_in1k':'vit_base_patch16_224.augreg_in1k',
'vit_base_patch16_clip_224.openai_ft_in12k_in1k':'vit_base_patch16_224.augreg_in1k',
'vit_base_patch16_clip_384.laion2b_ft_in12k_in1k':'vit_base_patch16_384.augreg_in1k',
'vit_base_patch16_clip_384.openai_ft_in12k_in1k':'vit_base_patch16_384.augreg_in1k',
'vit_base_patch32_clip_224.laion2b_ft_in12k_in1k':'vit_base_patch32_224.augreg_in1k',
'vit_base_patch32_clip_384.laion2b_ft_in12k_in1k':'vit_base_patch32_384.augreg_in1k',
'vit_base_patch32_clip_384.openai_ft_in12k_in1k':'vit_base_patch32_384.augreg_in1k',
 }

clip_ft_12k_df = df[df['Architecture'].isin(clip_ft_12k.keys())].sort_values(by=['Architecture']).reset_index(drop=True)
clip_ft_12k_bs_df = df[df['Architecture'].isin(clip_ft_12k.values())].sort_values(by=['Architecture']).reset_index(drop=True)
clip_ft_12k_df['Baseline_Architecture'] = clip_ft_12k_df.apply(lambda x: clip_ft_12k[x['Architecture']], axis=1)
clip_ft_12k_df_combined = combine_dfs(clip_ft_12k_df, clip_ft_12k_bs_df, repeat_bs=True)


In [46]:
# combine clip_ft_12k so that we have a baseline for each model
clip_ft_12k_df = df[df['Architecture'].isin(clip_ft_12k.keys())].sort_values(by=['Architecture']).reset_index(drop=True)
clip_ft_12k_bs_df = df[df['Architecture'].isin(clip_ft_12k.values())].sort_values(by=['Architecture']).reset_index(drop=True)
cols = ['Architecture', 'hAURC_climbing_01_TS', 'hAURC_climbing_01', 'hAURC_selective_01', 'Accuracy', 'Accuracy_TS', 'Gain_climbing_01_TS_F']
clip_ft_12k_combined = clip_ft_12k_df[cols]
clip_ft_12k_combined['Baseline_Architecture'] = clip_ft_12k_df.apply(lambda x: clip_ft_12k[x['Architecture']], axis=1)
# calculate Accuracy_improve: the difference between the accuracy of the model and its baseline.
# for that we need to get the baseline accuracy
for i,row in clip_ft_12k_combined.iterrows():
    bs_row = clip_ft_12k_bs_df[clip_ft_12k_bs_df['Architecture'] == row['Baseline_Architecture']]
    clip_ft_12k_combined.loc[i, 'Baseline_Accuracy'] = bs_row['Accuracy'].values[0]
    clip_ft_12k_combined.loc[i, 'Baseline_hAURC_selective_01'] = bs_row['hAURC_selective_01'].values[0]
    clip_ft_12k_combined.loc[i, 'Baseline_hAURC_climbing_01_TS'] = bs_row['hAURC_climbing_01_TS'].values[0]
    clip_ft_12k_combined.loc[i, 'Baseline_hAURC_climbing_01'] = bs_row['hAURC_climbing_01'].values[0]
    clip_ft_12k_combined.loc[i, 'Baseline_Gain_climbing_01_TS_F'] = bs_row['Gain_climbing_01_TS_F'].values[0]
    clip_ft_12k_combined['hAURC_improve_selective'] = 100*(clip_ft_12k_combined['Baseline_hAURC_selective_01'] - clip_ft_12k_combined['hAURC_selective_01'])/clip_ft_12k_combined['Baseline_hAURC_selective_01']
    clip_ft_12k_combined['hAURC_improve_climb_TS'] = 100*(clip_ft_12k_combined['Baseline_hAURC_climbing_01_TS'] - clip_ft_12k_combined['hAURC_climbing_01_TS'])/clip_ft_12k_combined['Baseline_hAURC_climbing_01_TS']
    clip_ft_12k_combined['hAURC_improve_climb'] = 100*(clip_ft_12k_combined['Baseline_hAURC_climbing_01'] - clip_ft_12k_combined['hAURC_climbing_01'])/clip_ft_12k_combined['Baseline_hAURC_climbing_01']
    clip_ft_12k_combined['Accuracy_improve'] = 100*(clip_ft_12k_combined['Accuracy_TS'] - clip_ft_12k_combined['Baseline_Accuracy'])/clip_ft_12k_combined['Baseline_Accuracy']
    clip_ft_12k_combined['Accuracy_delta'] = clip_ft_12k_combined['Accuracy_TS'] - clip_ft_12k_combined['Baseline_Accuracy']
    clip_ft_12k_combined['Gain_climb_improve'] = 100*(clip_ft_12k_combined['Gain_climbing_01_TS_F'] - clip_ft_12k_combined['Baseline_Gain_climbing_01_TS_F'])/clip_ft_12k_combined['Baseline_Gain_climbing_01_TS_F']

In [47]:
pretrained_12_models = ['coatnet_rmlp_1_rw2_224.sw_in12k_ft_in1k', 'coatnet_rmlp_2_rw_224.sw_in12k_ft_in1k', 'convnext_nano.in12k_ft_in1k',  'convnext_small.in12k_ft_in1k', 'convnext_tiny.in12k_ft_in1k', 'regnety_120.sw_in12k_ft_in1k',  'regnety_160.sw_in12k_ft_in1k',  'rexnetr_200.sw_in12k_ft_in1k', 'seresnextaa101d_32x8d.sw_in12k_ft_in1k']
pretrained_12_baselines = ['coatnet_rmlp_1_rw_224.sw_in1k', 'coatnet_rmlp_2_rw_224.sw_in1k', 'convnext_nano.d1h_in1k', 'convnext_small_torchvision', 'convnext_tiny_torchvision', 'regnety_120.pycls_in1k', 'regnety_160.pycls_in1k', 'rexnet_200.nav_in1k', 'seresnextaa101d_32x8d.ah_in1k']
len(pretrained_12_models), len(pretrained_12_baselines)

pretrained_12_df = df[df['Architecture'].isin(pretrained_12_models)].sort_values(by=['Architecture']).reset_index(drop=True)
pretrained_12_bs_df = df[df['Architecture'].isin(pretrained_12_baselines)].sort_values(by=['Architecture']).reset_index(drop=True)
pretrained_12_combined = combine_dfs(pretrained_12_df, pretrained_12_bs_df)

### SSL

In [48]:
ssl = {'resnet18.fb_ssl_yfcc100m_ft_in1k': 'resnet18.tv_in1k',
'resnet50.fb_ssl_yfcc100m_ft_in1k': 'resnet50.tv_in1k', 
'resnext50_32x4d.fb_ssl_yfcc100m_ft_in1k':  'resnext50_32x4d.tv_in1k',
'resnext101_32x4d.fb_ssl_yfcc100m_ft_in1k': 'resnext101_32x4d.gluon_in1k',
'resnext101_32x8d.fb_ssl_yfcc100m_ft_in1k': 'resnext101_32x8d.tv_in1k',
'resnet18.fb_swsl_ig1b_ft_in1k': 'resnet18.tv_in1k',
'resnet50.fb_swsl_ig1b_ft_in1k': 'resnet50.tv_in1k',
'resnext101_32x4d.fb_swsl_ig1b_ft_in1k': 'resnext101_32x4d.gluon_in1k',
'resnext101_32x8d.fb_swsl_ig1b_ft_in1k': 'resnext101_32x8d.tv_in1k',
'resnext50_32x4d.fb_swsl_ig1b_ft_in1k': 'resnext50_32x4d.tv_in1k',
'resnext101_32x8d.fb_wsl_ig1b_ft_in1k': 'resnext101_32x8d.tv_in1k'}
len(ssl)


11

In [49]:
# combine ssl so that we have a baseline for each model
ssl_df = df[df['Architecture'].isin(ssl.keys())].sort_values(by=['Architecture']).reset_index(drop=True)
ssl_bs_df = df[df['Architecture'].isin(ssl.values())].sort_values(by=['Architecture']).reset_index(drop=True)
cols = ['Architecture', 'hAURC_climbing_01_TS', 'hAURC_climbing_01', 'hAURC_selective_01', 'Accuracy', 'Accuracy_TS', 'Gain_climbing_01_TS_F']
ssl_combined = ssl_df[cols]
ssl_combined['Baseline_Architecture'] = ssl_df.apply(lambda x: ssl[x['Architecture']], axis=1)
# calculate Accuracy_improve: the difference between the accuracy of the model and its baseline.
# for that we need to get the baseline accuracy
for i,row in ssl_combined.iterrows():
    bs_row = ssl_bs_df[ssl_bs_df['Architecture'] == row['Baseline_Architecture']]
    ssl_combined.loc[i, 'Baseline_Accuracy'] = bs_row['Accuracy'].values[0]
    ssl_combined.loc[i, 'Baseline_hAURC_selective_01'] = bs_row['hAURC_selective_01'].values[0]
    ssl_combined.loc[i, 'Baseline_hAURC_climbing_01_TS'] = bs_row['hAURC_climbing_01_TS'].values[0]
    ssl_combined.loc[i, 'Baseline_hAURC_climbing_01'] = bs_row['hAURC_climbing_01'].values[0]
    ssl_combined.loc[i, 'Baseline_Gain_climbing_01_TS_F'] = bs_row['Gain_climbing_01_TS_F'].values[0]
    ssl_combined['hAURC_improve_selective'] = 100*(ssl_combined['Baseline_hAURC_selective_01'] - ssl_combined['hAURC_selective_01'])/ssl_combined['Baseline_hAURC_selective_01']
    ssl_combined['hAURC_improve_climb_TS'] = 100*(ssl_combined['Baseline_hAURC_climbing_01_TS'] - ssl_combined['hAURC_climbing_01_TS'])/ssl_combined['Baseline_hAURC_climbing_01_TS']
    ssl_combined['hAURC_improve_climb'] = 100*(ssl_combined['Baseline_hAURC_climbing_01'] - ssl_combined['hAURC_climbing_01'])/ssl_combined['Baseline_hAURC_climbing_01']
    ssl_combined['Accuracy_improve'] = 100*(ssl_combined['Accuracy_TS'] - ssl_combined['Baseline_Accuracy'])/ssl_combined['Baseline_Accuracy']
    ssl_combined['Accuracy_delta'] = ssl_combined['Accuracy_TS'] - ssl_combined['Baseline_Accuracy']
    ssl_combined['Gain_climb_improve'] = 100*(ssl_combined['Gain_climbing_01_TS_F'] - ssl_combined['Baseline_Gain_climbing_01_TS_F'])/ssl_combined['Baseline_Gain_climbing_01_TS_F']

ssl_combined['Method'] = 'SSL'

### Adversarial Training

In [50]:
adv = {'tf_efficientnet_b0.ap_in1k': 'tf_efficientnet_b0.in1k',
'tf_efficientnet_b1.ap_in1k': 'tf_efficientnet_b1.in1k',
'tf_efficientnet_b2.ap_in1k': 'tf_efficientnet_b2.in1k',
'tf_efficientnet_b3.ap_in1k': 'tf_efficientnet_b3.in1k',
'tf_efficientnet_b4.ap_in1k': 'tf_efficientnet_b4.in1k',
'tf_efficientnet_b5.ap_in1k': 'tf_efficientnet_b5.in1k',
'inception_resnet_v2.tf_ens_adv_in1k': 'inception_resnet_v2.tf_in1k',
'inception_v3.tf_adv_in1k': 'inception_v3.tf_in1k',}

In [51]:
# combine adv so that we have a baseline for each model
adv_df = df[df['Architecture'].isin(adv.keys())].sort_values(by=['Architecture']).reset_index(drop=True)
adv_bs_df = df[df['Architecture'].isin(adv.values())].sort_values(by=['Architecture']).reset_index(drop=True)
cols = ['Architecture', 'hAURC_climbing_01_TS', 'hAURC_climbing_01', 'hAURC_selective_01', 'Accuracy', 'Accuracy_TS', 'Gain_climbing_01_TS_F']
adv_combined = adv_df[cols]
adv_combined['Baseline_Architecture'] = adv_df.apply(lambda x: adv[x['Architecture']], axis=1)
# calculate Accuracy_improve: the difference between the accuracy of the model and its baseline.
# for that we need to get the baseline accuracy
for i,row in adv_combined.iterrows():
    bs_row = adv_bs_df[adv_bs_df['Architecture'] == row['Baseline_Architecture']]
    adv_combined.loc[i, 'Baseline_Accuracy'] = bs_row['Accuracy'].values[0]
    adv_combined.loc[i, 'Baseline_hAURC_selective_01'] = bs_row['hAURC_selective_01'].values[0]
    adv_combined.loc[i, 'Baseline_hAURC_climbing_01_TS'] = bs_row['hAURC_climbing_01_TS'].values[0]
    adv_combined.loc[i, 'Baseline_hAURC_climbing_01'] = bs_row['hAURC_climbing_01'].values[0]
    adv_combined.loc[i, 'Baseline_Gain_climbing_01_TS_F'] = bs_row['Gain_climbing_01_TS_F'].values[0]
    adv_combined['hAURC_improve_selective'] = 100*(adv_combined['Baseline_hAURC_selective_01'] - adv_combined['hAURC_selective_01'])/adv_combined['Baseline_hAURC_selective_01']
    adv_combined['hAURC_improve_climb_TS'] = 100*(adv_combined['Baseline_hAURC_climbing_01_TS'] - adv_combined['hAURC_climbing_01_TS'])/adv_combined['Baseline_hAURC_climbing_01_TS']
    adv_combined['hAURC_improve_climb'] = 100*(adv_combined['Baseline_hAURC_climbing_01'] - adv_combined['hAURC_climbing_01'])/adv_combined['Baseline_hAURC_climbing_01']
    adv_combined['Accuracy_improve'] = 100*(adv_combined['Accuracy_TS'] - adv_combined['Baseline_Accuracy'])/adv_combined['Baseline_Accuracy']
    adv_combined['Accuracy_delta'] = adv_combined['Accuracy_TS'] - adv_combined['Baseline_Accuracy']
    adv_combined['Gain_climb_improve'] = 100*(adv_combined['Gain_climbing_01_TS_F'] - adv_combined['Baseline_Gain_climbing_01_TS_F'])/adv_combined['Baseline_Gain_climbing_01_TS_F']

adv_combined['Method'] = 'Adversarial'

In [52]:
all_dfs = [df_dist_combined, pre_21_df_combined, clip_ft_1k_combined, clip_ft_12k_combined, pretrained_12_combined]
all_dfs_names = ['Distillation', 'Pretraining on Imagenet-21K', 'CLIP Training (finetuned on 1K)', 'CLIP Training (finetuned on 12K and 1K)', 'Pretraining on Imagenet-12K']
# make one big df but add a column for the name of the Method df
all_dfs = pd.concat(all_dfs, ignore_index=True)
all_dfs['Method'] = ''
all_dfs.loc[:len(df_dist_combined)-1, 'Method'] = f'Distillation ({len(df_dist_combined)})'
all_dfs.loc[len(df_dist_combined):len(df_dist_combined)+len(pre_21_df_combined)-1, 'Method'] = f'Pretraining on Imagenet-21K ({len(pre_21_df_combined)})'
all_dfs.loc[len(df_dist_combined)+len(pre_21_df_combined):len(df_dist_combined)+len(pre_21_df_combined)+len(clip_ft_1k_combined)-1, 'Method'] = f'CLIP Training (finetuned on 1K) ({len(clip_ft_1k_combined)})'
all_dfs.loc[len(df_dist_combined)+len(pre_21_df_combined)+len(clip_ft_1k_combined):len(df_dist_combined)+len(pre_21_df_combined)+len(clip_ft_1k_combined)+len(clip_ft_12k_combined)-1, 'Method'] = f'CLIP Training (finetuned on 12K and 1K) ({len(clip_ft_12k_combined)})'
all_dfs.loc[len(df_dist_combined)+len(pre_21_df_combined)+len(clip_ft_1k_combined)+len(clip_ft_12k_combined):, 'Method'] = f'Pretraining on Imagenet-12K ({len(pretrained_12_combined)})'

Unite Pretraining and CLIP

In [53]:
all_dfs = [df_dist_combined, pre_21_df_combined, clip_ft_1k_combined, clip_ft_12k_combined, pretrained_12_combined, ssl_combined, adv_combined]

pretrained_combined = pd.concat([pre_21_df_combined, pretrained_12_combined], ignore_index=True)
clip_combined = pd.concat([clip_ft_1k_combined, clip_ft_12k_combined], ignore_index=True)
df_dist_combined['Method'] = f'Distillation ({len(df_dist_combined)})'
pretrained_combined['Method'] = f'Pretraining ({len(pretrained_combined)})'
clip_combined['Method'] = f'CLIP ({len(clip_combined)})'
ssl_combined['Method'] = f'SSL ({len(ssl_combined)})'
adv_combined['Method'] = f'Adversarial ({len(adv_combined)})'
all_dfs = [df_dist_combined, pretrained_combined, clip_combined, ssl_combined, adv_combined]
all_dfs = pd.concat(all_dfs, ignore_index=True)
all_dfs.to_csv('all_dfs.csv', index=False)

In [54]:
all_dfs = pd.read_csv('all_dfs.csv')

## Section 5 Plot

In [55]:
import plotly.express as px
import plotly.graph_objects as go

In [56]:
delta = {
    'Knowledge Distillation': 4.5,
    'Pretraining (ImageNet21k or ImageNet12k)': 4.5,
    'Contrastive Language-Image Pretraining (CLIP)': -1,
    'Semi-Supervised Learning': 5.5,
    'Adversarial Training': 1.2
}

In [57]:
y_axis = 'hAURC Improvement over Baseline (%)'
legend_x_cord, legend_y_cord = 0, 10.0
full_results_df = all_dfs
full_results_df = full_results_df.rename(columns={'hAURC_improve_climb_TS': 'hAURC Improvement over Baseline (%)'})
medians = pd.DataFrame(full_results_df[['Method', y_axis]].groupby('Method')[y_axis].median().reset_index())
medians['Num_Models'] = medians['Method'].apply(lambda x: x.split("(")[1].strip())
medians['Method'] = medians['Method'].apply(lambda x: x.split("(")[0].strip())
medians['Method'] = medians['Method'].replace({'Distillation': 'Knowledge Distillation', 'Pretraining': 'Pretraining (ImageNet21k or ImageNet12k)', 'CLIP': 'Contrastive Language-Image Pretraining (CLIP)', 'SSL': 'Semi-Supervised Learning', 'Adversarial': 'Adversarial Training'})
full_results_df['Method'] = full_results_df['Method'].apply(lambda x: x.split("(")[0].strip())
full_results_df['Method'] = full_results_df['Method'].replace({'Distillation': 'Knowledge Distillation', 'Pretraining': 'Pretraining (ImageNet21k or ImageNet12k)', 'CLIP': 'Contrastive Language-Image Pretraining (CLIP)', 'SSL': 'Semi-Supervised Learning', 'Adversarial': 'Adversarial Training'})
y_axis_range = full_results_df[y_axis].to_numpy()
range_y = [min(y_axis_range) - 0.1, max(y_axis_range) + 0.1]
fig = px.box(full_results_df.dropna(subset=['Method']), x="Method", y=y_axis, hover_name="Architecture",
                color='Method', points="all",
                category_orders={"Method": ['a','b']})
# make font larger
fig.update_xaxes(title='', tickmode='array',
                tickvals=[0,1,2,3,4,5,6],
                ticktext=['a','b','Knowledge</br></br>Distillation', 'Pretraining</br></br>(ImageNet21k or</br>ImageNet12k)', 'Contrastive </br></br> Language-Image</br>Pretraining(CLIP)', 'Semi-Supervised</br></br>Learning', 'Adversarial</br></br>Training'])
fig.update_yaxes(title_text=y_axis, title_font=dict(size=34))

font_size = 33
fig.update_layout(
    font=dict(
        size=font_size,
    ))
# don't show legend 
fig.update_layout(showlegend=False)

# Adding borders (frame) to the figure
fig.update_layout(shapes=[
    # unfilled rectange
    go.layout.Shape(type="rect", xref="paper", yref="paper",
                    x0=0, y0=0, x1=1.0, y1=1.0, line={'width': 1, 'color': 'black'}), ])
fig.update_layout(plot_bgcolor='rgba(0,0,0,0)')  # Removing graph background color
# fig.update_xaxes(showgrid=True, gridwidth=0.5, gridcolor='LightGrey')  # Modifying grids color and opacity
fig.update_yaxes(showgrid=True, gridwidth=0.5, gridcolor='LightGrey')  # Modifying grids color and opacity
# snap the margins around the border
fig.update_layout(margin=dict(l=0, r=5, t=60, b=10))
fig.add_hline(y=0)
fig.write_html('results/figures/methods_comparison.html')
fig.write_image('results/figures/methods_comparison.pdf', width=1920, height=800)